# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

Answer:

If the input of the produced are the features of the students then the output should be whether the student will pass. As a result, this should be a classification problem.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns
student_data.head()

Student data read successfully!


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns)-1
n_passed = len(student_data[student_data['passed']=='yes'])
n_failed = len(student_data[student_data['passed']=='no'])
grad_rate = n_passed/float(n_passed+n_failed)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate*100)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [4]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [5]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))
X_all.head()

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,1.0,0.0,1.0,0.0,18,0.0,1.0,1.0,0.0,1.0,...,1,0,0,4,3,4,1,1,3,6
1,1.0,0.0,1.0,0.0,17,0.0,1.0,1.0,0.0,0.0,...,1,1,0,5,3,3,1,1,3,4
2,1.0,0.0,1.0,0.0,15,0.0,1.0,0.0,1.0,0.0,...,1,1,0,4,3,2,2,3,3,10
3,1.0,0.0,1.0,0.0,15,0.0,1.0,1.0,0.0,0.0,...,1,1,1,3,2,2,1,1,5,2
4,1.0,0.0,1.0,0.0,16,0.0,1.0,1.0,0.0,0.0,...,1,0,0,4,3,2,1,2,5,4


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [6]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train
# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
student_data_shuffled = student_data.sample(frac=1)
X_train = preprocess_features(student_data_shuffled[:num_train][feature_cols])
y_train = student_data_shuffled[:num_train][target_col]
X_test = preprocess_features(student_data_shuffled[num_train:][feature_cols])
y_test = student_data_shuffled[num_train:][target_col]
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [7]:
df = pd.DataFrame(columns = ['Training set size', 'Method', 'Train time', 'Test time', 'F1 score for train set',
                         'F1 score for test set'] )

In [8]:
# Train a model
import time
def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)
    train_time = "{:.3f}".format(end - start)
    return float(train_time)

# TODO: Choose a model, import it and instantiate an object
from sklearn.naive_bayes import MultinomialNB, GaussianNB
clf = MultinomialNB()

print clf
# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Training MultinomialNB...
Done!
Training time (secs): 0.021


0.021

In [9]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score, make_scorer, accuracy_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    
    return f1_score(target.values, y_pred, pos_label='yes'), float("{:.3f}".format(end - start))

train_f1_score, _ = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.788461538462


In [10]:
# Predict on test data
test_f1, _ = predict_labels(clf, X_test, y_test)
print "F1 score for test set: {}".format(test_f1)

Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.777777777778


In [11]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_time = train_classifier(clf, X_train, y_train)
    F1_train, _ = predict_labels(clf, X_train, y_train)
    F1_test, test_time = predict_labels(clf, X_test, y_test)
    print "F1 score for training set: {}".format(F1_train)
    print "F1 score for test set: {}".format(F1_test)
    return F1_train, F1_test, train_time, test_time


## Naive Bayes
Answer: 

- What are the general applications of this model? What are its strengths and weaknesses?

Naive Bayes Classification is suitable for classification of nominal values.  For example, text classification and document categories.

Strengths: 1. Naive Bayes classifiers are easy to implement and apply 2. The performance of NB classifier will not be seriously affect by high dimensionality 3. The speed of training and testing is relatively fast

Weaknesse: 1. If the features are highly dependent, the assumption when using NB classifier that the features are independent will cause poor performance of this model. 2. The maximum likelihood may cause overfitting on training set. 3. The performance is sensitive to the quality of the data preparation.
- Given what you know about the data so far, why did you choose this model to apply?

In this project, MultinomialNB is used as a classifier. The reason for choosing this Naive Bayes it that there is no strong and obvious connection and/or dependence among the features. As a result, we can assume that the features are independent, though some of the features may have some connection in real life. The reason for choosing Multinomial module rather than  GaussianNB is that date set mainly contains discreet features, which is not suiable for GaussianNB. Also the number of features of this data set is 30, which is relatively large number of features. Since NB classifier is good with high dimensionality, this is also a reason for choosing it.

In [12]:
# TODO: Run the helper function above for desired subsets of training data
train_sizes = [100, 200, 300]
for size in train_sizes:
#     X_train = preprocess_features(student_data_shuffled[:size][feature_cols])
#     y_train = student_data_shuffled[:size][target_col]
    F1_train, F1_test, train_time, test_time = train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
    line = {
        'Training set size':size,
        'Method':clf.__class__.__name__,
        'F1 score for train set':F1_train,
        'F1 score for test set':F1_test,
        'Train time':train_time,
        'Test time':test_time
    }
    df = df.append(line, ignore_index=True)
df[df['Method']==clf.__class__.__name__]

------------------------------------------
Training set size: 100
Training MultinomialNB...
Done!
Training time (secs): 0.002
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.761194029851
F1 score for test set: 0.769230769231
------------------------------------------
Training set size: 200
Training MultinomialNB...
Done!
Training time (secs): 0.002
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.770318021201
F1 score for test set: 0.769230769231
------------------------------------------
Training set size: 300
Training MultinomialNB...
Done!
Training time (secs): 0.001
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000
Predicting labels using MultinomialNB...
Done!
Prediction time (secs): 0.000


,Training set size,Method,Train time,Test time,F1 score for train set,F1 score for test set
0,100.0,MultinomialNB,0.002,0.000,0.761194,0.769231
1,200.0,MultinomialNB,0.002,0.001,0.770318,0.769231
2,300.0,MultinomialNB,0.001,0.000,0.788462,0.777778


## Decision Tree Classifier
Answer:

- What are the general applications of this model? What are its strengths and weaknesses?
General Application: Decision Tree Classifier is suitable for numerical and nominal value regression and/or classification.

Strengths: 1. Decision Tree Classifier has insight in the data and easy to be understand by humans. 2. Not expensive to compute. 3. Robust to missing data

Weaknesses: Easy to have overfitting problem.

- Given what you know about the data so far, why did you choose this model to apply?

This is a classification problem with nominal data. In fact, it is also fair to say it's a classification problem with nominal and numerical data, because some of the features, like absences, can be treat as numerical features(the value matters). There are 30 features in the data set, which is not too large for DT Classifier. As a result, Decision tree classifier should be suitable for this application.

In [13]:
# TODO: Train and predict using two other models
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
train_sizes = [100, 200, 300]
for size in train_sizes:
#     X_train = preprocess_features(student_data_shuffled[:size][feature_cols])
#     y_train = student_data_shuffled[:size][target_col]
    F1_train, F1_test, train_time, test_time = train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
    line = {
        'Training set size':size,
        'Method':clf.__class__.__name__,
        'F1 score for train set':F1_train,
        'F1 score for test set':F1_test,
        'Train time':train_time,
        'Test time':test_time
    }
    df = df.append(line, ignore_index=True)
df[df['Method']==clf.__class__.__name__]

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
F1 score for test set: 0.610169491525
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
F1 score for test set: 0.661016949153
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.003
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
Predicting labels using Dec

,Training set size,Method,Train time,Test time,F1 score for train set,F1 score for test set
3,100.0,DecisionTreeClassifier,0.001,0.0,1.0,0.610169
4,200.0,DecisionTreeClassifier,0.002,0.0,1.0,0.661017
5,300.0,DecisionTreeClassifier,0.003,0.0,1.0,0.734375


## k-Nearest Neighbors
- What are the general applications of this model? What are its strengths and weaknesses?
KNN method can be applied to classification and regression with numeric values and nominal values.

Strengths: As KNN method only refer to some nearest neighbors, the result will not be affected by outliers. Also, KNN makes no assumptions about data.

Cons: KNN requires high computation and memory resources.

- Given what you know about the data so far, why did you choose this model to apply?

As this is a classification problem with mixed numeric and nominal values, KNN may be a suitable model for it.

In [14]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
train_sizes = [100, 200, 300]
for size in train_sizes:
#     X_train = preprocess_features(student_data_shuffled[:size][feature_cols])
#     y_train = student_data_shuffled[:size][target_col]
    F1_train, F1_test, train_time, test_time = train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)

    line = {
        'Training set size':size,
        'Method':clf.__class__.__name__,
        'F1 score for train set':F1_train,
        'F1 score for test set':F1_test,
        'Train time':train_time,
        'Test time':test_time
    }
    df = df.append(line, ignore_index=True)
df[df['Method']==clf.__class__.__name__]

------------------------------------------
Training set size: 100
Training KNeighborsClassifier...
Done!
Training time (secs): 0.002
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.003
F1 score for training set: 0.785714285714
F1 score for test set: 0.715328467153
------------------------------------------
Training set size: 200
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.004
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.860927152318
F1 score for test set: 0.791666666667
------------------------------------------
Training set size: 300
Training KNeighborsClassifier...
Done!
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Done!
Prediction time (secs): 0.009
Predicting labels usi

,Training set size,Method,Train time,Test time,F1 score for train set,F1 score for test set
6,100.0,KNeighborsClassifier,0.002,0.003,0.785714,0.715328
7,200.0,KNeighborsClassifier,0.001,0.002,0.860927,0.791667
8,300.0,KNeighborsClassifier,0.001,0.003,0.856481,0.813793


## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [15]:
df
# TODO: Fine-tune your model and report the best F1 score

,Training set size,Method,Train time,Test time,F1 score for train set,F1 score for test set
0,100.0,MultinomialNB,0.002,0.000,0.761194,0.769231
1,200.0,MultinomialNB,0.002,0.001,0.770318,0.769231
2,300.0,MultinomialNB,0.001,0.000,0.788462,0.777778
3,100.0,DecisionTreeClassifier,0.001,0.000,1.000000,0.610169
4,200.0,DecisionTreeClassifier,0.002,0.000,1.000000,0.661017
5,300.0,DecisionTreeClassifier,0.003,0.000,1.000000,0.734375
6,100.0,KNeighborsClassifier,0.002,0.003,0.785714,0.715328
7,200.0,KNeighborsClassifier,0.001,0.002,0.860927,0.791667
8,300.0,KNeighborsClassifier,0.001,0.003,0.856481,0.813793


- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?

From the table above we can figure out that the Decision Tree Classifier is prone to overfitting, because the train set F1 score is always 1. Also, the test set F1 score is poor comparing to the other two classifiers.
The MultinomialNB classifier performance is better than DecisionTreeClassifier: when the training set size achieved 300, the difference of F1 score between test set and train set is decreased to 0.788-0.778=0.01, which indicates it is robust to overfitting. The KNeighbors classifier's performance is the best out of the three classifiers. The F1 score of test set is the highest when training set size achieved 300.

DecisionTreeClassifier's performance is poor comparing to the other two.  MultinomialNB's performance is decent in terms of resource cost and robust to overfitting. However, this size of the whole data set is 395, the wrost case of time costing for KNeighborsClassifier is 0.001s(training time), 0.003s(predicting time), which is not as good as MultinomialNB but this is still a trivial cost. Most importantly, the testing F1 score is among the all three and KNN method is closer to human intuition, which is easy to interpret and tuning. As a result, KNeighborsClassifier is chosen to be the most suitable model for this project.

- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).

KNN model is finally chosen to be the model for this project. The mechanism is close to human intuition: 
this model works by comparing the new incoming input, which is the case we need to predict, with the existing records. The model will compare the features of the new input and the existing records and find out the k most similar samples from the existing records. The features here are the information we obtained about those students, for example, their sex, school and age. The similarity can be discribed as how close their features are and this can be calculated as the numerical difference. After that, k most similar samples from the existing cases will be selected to vote that the new input case should be classified into the class they belong to. The class here is whether the student passes the exam. Sometimes, we can assign more weight to the samples closer to the input case in the voting process. In the end, the input case will be assigned to most voted class. 

- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

The final F1 score is 0.854.

In [16]:
from sklearn import grid_search
from sklearn import metrics
knn = KNeighborsClassifier()
scorer = metrics.make_scorer(f1_score, pos_label='yes')
parameters = {'leaf_size':[30,40,50,60], 
              'weights':['uniform','distance'],
             'algorithm' : ['ball_tree', 'kd_tree','brute'],
              'n_neighbors':[3,5,10,20,30],
             'p':[1,2,3],
             }
clf_GS = grid_search.GridSearchCV(knn, parameters, scoring = scorer)
clf_GS.fit(X_train, y_train)
print clf_GS.best_params_
print clf_GS.score(X_test, y_test)

{'n_neighbors': 20, 'weights': 'distance', 'leaf_size': 30, 'algorithm': 'brute', 'p': 1}
0.853503184713


/Users/zhuochen/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:418: ChangedBehaviorWarning: The long-standing behavior to use the estimator's score function in GridSearchCV.score has changed. The scoring parameter is now used.
  ChangedBehaviorWarning)
